# VisB


Initially VisB was developed for [[ProB2-UI]], the new user interface of ProB based on JavaFX and the ProB2-Java-API.
It is included in ProB2-UI version 1.1.0 and later, which [[Download#ProB2-UI using Java FX|can be downloaded here]].
Users of ProB2-UI version 1.0.0 can [[#Installing the VisB plugin for older ProB2-UI versions|install VisB as a plugin]].

[[File:ProB2UI_VisB_View.png|center|550px]]

ProB Tcl/Tk and probcli version 1.11.0 and later also support VisB visualisations:

* with <tt>probcli File.mch ... -visb VISBJSONFILE HTMLFILE</tt> you can export the animator's history to a self-contained HTML file including a visualisation of the states in the history
* in ProB Tcl/Tk you can visualise the current state or the history using a VisB visualisation file (it is shown in an external browser) by right clicking in the "State Properties" or the "History" pane.
Hovers work in both cases, but unlike in ProB2-UI, you cannot click to perform events.

An article about VisB has been [https://rdcu.be/b4rqh published in the ABZ'2020 proceedings].


As of version 1.12 you can also provide many of the VisB annotations via B DEFINITIONS.
This can be done in addition to a VisB JSON file or completely replacing the JSON file.

By adding this definition to your B machine you specify that you wish to use an empty
SVG file as base:
<pre>
  VISB_JSON_FILE == "";
</pre>

With a VISB_SVG_BOX definition you can set the size of the generated empty SVG file:
<pre>
  VISB_SVG_BOX == rec(height:formula, width:1800);
</pre>

Note: you can also provide an optional <tt>viewBox</tt> attribute as a B string.

You can populate the empty SVG file with your own new objects using DEFINITIONS that start with <pre>VISB_SVG_OBJECTS</pre>.
Here is an example where we create one red circle:

In [12]:
::load
MACHINE test1
DEFINITIONS
  VISB_JSON_FILE == "";
  VISB_SVG_BOX == rec(height:100, width:400);
  VISB_SVG_OBJECTS == rec(`id`:"button", svg_class:"circle",
     cx:200,cy:50, r:30, fill: "red",
     stroke:"black", `stroke-width`:2);
END

Loaded machine: test1

In [13]:
:init

Executed operation: INITIALISATION()

In [14]:
:show

<VisB visualization>

A VISB_SVG_OBJECT definition should either define a record or a set of records. The fields of the record specify the kind of object created and its initial attributes:
* usually all created object should have an id (so that you can later modify the attributes)
* you have to provide an <tt>svg_class</tt> attribute: circle, ellipse, line, path, polygon, polyline, rect, text
* you then can (and probably) should provide necessary attributes and initial values for your object

Note: <tt>id</tt> is a keyword in B and hence you have to use backquotes around it. Similarly, B does not allow identifiers with hyphens, hence you need surround such attributes (like <tt>stroke-width</tt>) with backquotes as well.

In [27]:
::load
MACHINE test2
DEFINITIONS
  N == 5; WID==400.0;
  VISB_JSON_FILE == "";
  VISB_SVG_BOX == rec(height:100, width:WID+30.0);
  VISB_SVG_OBJECTS == UNION(i).(i:1..N|
    {rec(`id`:("circ",i), svg_class:"circle",
     cx:real(i)*WID/real(N),cy:50, r:(30-i), fill: "red",
     stroke:"black", `stroke-width`:2)});
END

Loaded machine: test2

In [28]:
:init

Executed operation: INITIALISATION()

In [29]:
:show

<VisB visualization>

In [30]:
:table VISB_SVG_OBJECTS

|cx|cy|fill|id|r|stroke|stroke-width|svg_class|
|---|---|---|---|---|---|---|---|
|80.0|50|"red"|("circ"↦1)|29|"black"|2|"circle"|
|160.0|50|"red"|("circ"↦2)|28|"black"|2|"circle"|
|240.0|50|"red"|("circ"↦3)|27|"black"|2|"circle"|
|320.0|50|"red"|("circ"↦4)|26|"black"|2|"circle"|
|400.0|50|"red"|("circ"↦5)|25|"black"|2|"circle"|
